# Working with AWS S3


## Purpose
This notebook is desinged for AWS developers to play around and explore different ways to interact with AWS S3.


# Setup

## Import Libraries
We import all the required Python libraries.

In [ ]:
#!pip install shortuuid

In [1]:
import logging
import shortuuid
import boto3
from botocore.exceptions import ClientError

In [2]:
boto3.__version__

'1.22.9'

In [3]:
#Determine which credentials(profile) to use embeded in .aws/config file.
boto3 = boto3.session.Session(profile_name='AWS-DEMO')

In [4]:
s3 = boto3.resource('s3')

## Create S3 buckets


In [5]:
# Generate a bucket name consisting of an arbitrary prefix and a random string.
def create_bucket_name(bucket_prefix):
    return '-'.join([bucket_prefix, str.lower(shortuuid.uuid())])

In [6]:
# The default region would be us-east-1(N.Virgina) unless specified.
def create_bucket(bucket_prefix, region=None):
    # Create bucket
    bucket_name = create_bucket_name(bucket_prefix)
    try:
        if region is None:
            s3 = boto3.resource('s3')
            s3.create_bucket(Bucket=bucket_name)
        elif region == "us-east-1":
            s3 = boto3.resource('s3')
            s3.create_bucket(Bucket=bucket_name)        
        else:
            s3 = boto3.resource('s3', region_name=region)
            location = {'LocationConstraint': region}
            s3.create_bucket(Bucket=bucket_name,
                             CreateBucketConfiguration=location)
    except ClientError as e:
        logging.error(e)
        return False
    return True


In [7]:
# Create a bucket with your favorite prefix. And optionally, specify a region.
# Your buckets will be created in the us-east-1 region otherwise.
create_bucket("chameleon")
create_bucket("godzilla", region="us-west-2")

True

In [8]:
# List all buckets – Easy Way (resource-API way)
buckets = []
for bucket in s3.buckets.all():
    buckets.append(bucket.name)
    print(bucket.name)

aws-cloudtrail-logs-465968594676-ace7d89f
cf-temps
chameleon-8nxvf2bihsjkeisz5cctze
config-bucket-465968594676
dev-course-4
dev-course-4-pollynotesapi-1it5qucgww02t
dev-course-4-pollynotesweb-1lo5f3h0y29si
dragon-lkj4rn23
godzilla-2swbij7kajexowexanw9p3
input-aksfj324lk5324lkjsdfl
output-ksdflkjwer23421
thebestfoodever
thebestfoodever-australia


In [9]:
my_bucket = [i for i in buckets if i.startswith('godzilla-')]
my_bucket = my_bucket[0]
my_bucket

'godzilla-2swbij7kajexowexanw9p3'

In [10]:
s3.BucketVersioning(my_bucket).enable()

{'ResponseMetadata': {'RequestId': '92WVDCK24W4QC0RD',
  'HostId': 'Z75l8uyE0Zn68828gfic8863hCs/mg5GqKwNGhN023HZFHnS0lVUkvMyWK9FqvTlkbro+sDXDSo=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'Z75l8uyE0Zn68828gfic8863hCs/mg5GqKwNGhN023HZFHnS0lVUkvMyWK9FqvTlkbro+sDXDSo=',
   'x-amz-request-id': '92WVDCK24W4QC0RD',
   'date': 'Tue, 19 Jul 2022 19:16:04 GMT',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 1}}

In [11]:
# Create a list of all buckets' information and print the response. (hard way or client-api way)
response = s3.meta.client.list_buckets()
response

{'ResponseMetadata': {'RequestId': '53JSR4S8G5FKG1WF',
  'HostId': 'PgsQOoGpBhGHyIXo9nRlh9O4kMUvpp3Ja4JzpQRYaYWA0CsFE8OjVibT5aOTxp11bOt+joQSyD4=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'PgsQOoGpBhGHyIXo9nRlh9O4kMUvpp3Ja4JzpQRYaYWA0CsFE8OjVibT5aOTxp11bOt+joQSyD4=',
   'x-amz-request-id': '53JSR4S8G5FKG1WF',
   'date': 'Tue, 19 Jul 2022 19:17:21 GMT',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Buckets': [{'Name': 'aws-cloudtrail-logs-465968594676-ace7d89f',
   'CreationDate': datetime.datetime(2022, 2, 21, 13, 57, 28, tzinfo=tzutc())},
  {'Name': 'cf-temps',
   'CreationDate': datetime.datetime(2022, 2, 21, 14, 16, 36, tzinfo=tzutc())},
  {'Name': 'chameleon-8nxvf2bihsjkeisz5cctze',
   'CreationDate': datetime.datetime(2022, 7, 19, 19, 14, 48, tzinfo=tzutc())},
  {'Name': 'config-bucket-465968594676',
   'CreationDate': datetime.datetime(2022, 2, 21, 14, 13, 4, tzinfo=tzutc())},
  {'Name':

In [12]:
# Extract and list bucket names
for bucket in response['Buckets']:
    print(f'{bucket["Name"]}')

aws-cloudtrail-logs-465968594676-ace7d89f
cf-temps
chameleon-8nxvf2bihsjkeisz5cctze
config-bucket-465968594676
dev-course-4
dev-course-4-pollynotesapi-1it5qucgww02t
dev-course-4-pollynotesweb-1lo5f3h0y29si
dragon-lkj4rn23
godzilla-2swbij7kajexowexanw9p3
input-aksfj324lk5324lkjsdfl
output-ksdflkjwer23421
thebestfoodever
thebestfoodever-australia


## Create some files

In [13]:
import os
if not os.path.exists('./tmp/'):
    os.makedirs('./tmp/')

In [14]:
def create_temp_file(size, file_name, file_content, extention=".txt", path='./tmp/'):
    random_file_name = ''.join([file_name, shortuuid.uuid(), extention])
    with open(path + random_file_name, 'w') as temp:
        temp.write(str(file_content) * size)
    return random_file_name

In [15]:
KB = 1024
MB = KB * KB
temp_file = create_temp_file(200 * KB, 'temp', 'xyz', '.mp4')

## Upload files

In [16]:
def upload_file(bucket, file_name, path='./tmp/'):
    
    # Upload a file
    s3 = boto3.resource('s3')
    try:
        obj = s3.Object(bucket, file_name)
        obj.upload_file(Filename=path + file_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [17]:
upload_file(my_bucket, temp_file)

True

In [18]:
bucket = s3.Bucket(my_bucket)
for obj in bucket.objects.all():
    print(obj.key)

tempTiQkGynqMXXjXoi4wnd3UZ.mp4


## Upload Files (Encryption Enabled)

In [19]:
def upload_file(bucket, file_name, path='./tmp/'):
    
    # Upload a file
    s3 = boto3.resource('s3')
    try:
        obj = s3.Object(bucket, file_name)
        obj.upload_file(Filename=path + file_name,
                        ExtraArgs={
                            'ServerSideEncryption': 'AES256'})
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [20]:
upload_file(my_bucket, temp_file)

True

In [21]:
bucket = s3.Bucket(my_bucket)
for obj in bucket.objects.all():
    print(obj.key)

tempTiQkGynqMXXjXoi4wnd3UZ.mp4


## Multi-Part Upload

In [22]:
from boto3.s3.transfer import TransferConfig

In [23]:
KB = 1024
MB = KB * KB
config = TransferConfig(multipart_threshold=4 * MB, 
                        max_concurrency=10,
                        multipart_chunksize=4 * MB,
                        use_threads=True)

In [24]:
def multipart_upload_file(bucket, file_name, path='./tmp/'):
    
    # Upload the file
    s3 = boto3.resource('s3')
    try:
        obj = s3.Object(bucket, file_name)
        obj.upload_file(Filename=path + file_name,
                        Config=config,
                        ExtraArgs={
                            'ServerSideEncryption': 'AES256'})
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [25]:
KB = 1024
MB = KB * KB
large_file = create_temp_file(10 * MB, 'temp', 'xyz', '.mp4')

In [26]:
upload_file(my_bucket, large_file)

True

In [27]:
bucket = s3.Bucket(my_bucket)
for obj in bucket.objects.all():
    print(obj.key)

tempKVbv54ZGm6XRTwvyiM3E8D.mp4
tempTiQkGynqMXXjXoi4wnd3UZ.mp4


## Delete all S3 buckets and Objects inside

In [28]:
def delete_all_objects(bucket_name):
    res = []
    bucket=s3.Bucket(bucket_name)
    for obj_version in bucket.object_versions.all():
        res.append({'Key': obj_version.object_key,
                    'VersionId': obj_version.id})
#    print(res)
    bucket.delete_objects(Delete={'Objects': res})

In [29]:
delete_all_objects(my_bucket)
s3.Bucket(my_bucket).delete()

{'ResponseMetadata': {'RequestId': 'WXSP9TMYW7RMRY7R',
  'HostId': 'AyJAybTP1RzCpO+Td1iko2X/erUUVcdI2NTsW3PcwwAjDUYvCcgWKK494FfCdOjKqqnetfueY9s=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'AyJAybTP1RzCpO+Td1iko2X/erUUVcdI2NTsW3PcwwAjDUYvCcgWKK494FfCdOjKqqnetfueY9s=',
   'x-amz-request-id': 'WXSP9TMYW7RMRY7R',
   'date': 'Tue, 19 Jul 2022 19:23:20 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

In [ ]:
#for bucket in s3.buckets.all():
#    delete_all_objects(bucket.name)
#    s3.Bucket(bucket.name).delete()